# Setting Up

In [ ]:
!nvidia-smi

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

! pip install -q transformers[sentencepiece] fastbook fastai ohmeow-blurr nbdev

import torch
from transformers import AutoModelForSequenceClassification, AutoConfig
from fastai.text.all import *
from blurr.text.data.all import *
from blurr.text.modeling.all import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/blurr/text/modeling/question_answering.py:31: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  squad_metric = load_metric("squad")


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
%cd /content/drive/MyDrive/Colab Notebooks/NLP train

/content/drive/MyDrive/Colab Notebooks/dokkho NLP test


# Data processing:

initial test

In [ ]:
import pandas as pd
qdf = pd.read_csv('quote-nlp-dataset-scraped.csv', encoding='utf-8', engine='python')#, on_bad_lines='skip', sep='delimiter')
print(qdf.shape)

qdf.head()

(38560, 6)


,attirbuted author,quote,quote_info_list,quote_info_list_w_author,revised_quote_info,tag_cat_list
0,Albert Einstein,Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.,"['attributed-no-source', 'human-nature', 'humor', 'infinity', 'philosophy', 'science', 'stupidity', 'universe', 'Albert Einstein']","['attributed-no-source', 'human-nature', 'humor', 'infinity', 'philosophy', 'science', 'stupidity', 'universe', 'Albert Einstein']","['humor', 'philosophy', 'science']","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,Frank Zappa,"So many books, so little time.","['books', 'humor', 'Frank Zappa']","['books', 'humor', 'Frank Zappa']","['books', 'humor']","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,Mae West,"You only live once, but if you do it right, once is enough.","['humor', 'life', 'Mae West']","['humor', 'life', 'Mae West']","['humor', 'life']","[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,Narcotics Anonymous,"Insanity is doing the same thing, over and over again, but expecting different results.","['humor', 'insanity', 'life', 'misattributed-ben-franklin', 'misattributed-mark-twain', 'misattributed-to-einstein', 'Narcotics Anonymous']","['humor', 'insanity', 'life', 'misattributed-ben-franklin', 'misattributed-mark-twain', 'misattributed-to-einstein', 'Narcotics Anonymous']","['humor', 'life']","[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,Jane Austen,"The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.","['aliteracy', 'books', 'classic', 'humor', 'Jane Austen']","['aliteracy', 'books', 'classic', 'humor', 'Jane Austen']","['books', 'humor']","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [ ]:
nan_df = qdf[qdf.isna().any(axis=1)]

# drop even if single nan col in rows
print(qdf.shape)
qdf.dropna(inplace= True)
print('after dropping nan rows:', qdf.shape)
#display(qdf)

#drop duplicates. keep only one occurence.
qdf.drop_duplicates(subset="quote", keep='first', inplace=True)
print('after dropping duplicate rows:', qdf.shape)

(39751, 4)
after dropping nan rows: (39751, 4)
after dropping duplicate rows: (39751, 4)


# Scrap error cleaning:

load data:

In [ ]:
qdf = pd.read_csv('quote_file.csv', encoding='utf-8', engine='python')
qdf.shape

(39741, 4)

find error:

In [ ]:
info_list = qdf.quote_info_list.to_list()

scrap_error_idx_list = []
for tag_list in info_list: #info_list is string. check if this string is python list or not

    try:
        #print(tag_list)
        tag_list = eval(tag_list)
        #print(type(tag_list), tag_list)
    except :
        idx = qdf[ qdf['quote_info_list'] == tag_list ].index.values.astype(int)[0] #qdf[ qdf['quote_info_list_w_author'] == info_list ].index
        #print(idx)
        #display(qdf.iloc[idx])
        scrap_error_idx_list.append(idx)


print(len(scrap_error_idx_list))
#must descend order before delete
scrap_error_idx_list.sort(reverse=True) #must so that you drop from under keeping serial intact
print(scrap_error_idx_list)

0
[]


view:

In [ ]:
for idx in scrap_error_idx_list:
    print(qdf.iloc[idx],'\n')

drop:

In [ ]:
#print(scrap_error_idx_list)

print(qdf.shape)
#drop on index
for idx in scrap_error_idx_list:
    #try:
        print(idx)
        qdf.drop( [qdf.index[idx]], inplace=True,)
    #except:
        #print(qdf.iloc[idx],'\n')

print('after:', qdf.shape)


(39742, 4)
20775
after: (39741, 4)


save:

In [ ]:
print(qdf.shape)
qdf.to_csv('quote-nlp-dataset_cleaned.csv',encoding='utf-8-sig', index=False) #encoding='utf-8-sig' should work. also encoding='utf-16' but does not alawys work

(38560, 5)


In [ ]:
qdf = pd.read_csv('quote-nlp-dataset_cleaned.csv', encoding='utf-8', engine='python')
print(qdf.shape)
qdf.tail(100)

Scrap error Cleaned.

In [ ]:
info_list = qdf.quote_info_list.to_list()
info_list

tag_count = {}

for tag_list in info_list: # type(info_list) is string
    tag_list = eval(tag_list)
    print(tag_list)
    for tag in tag_list:
        if tag in tag_count.keys():
            tag_count[tag] += 1
        else:
            tag_count[tag] = 1
print(f"Number of tags: {len(tag_count)}")
print(tag_count)        # irbz vai Number of Genres: 640 for 6k. for my 39751 keep 4000


view count. sorted:

In [ ]:
# #view sorted count
# import numpy as np
# keys = list(tag_count.keys())
# values = list(tag_count.values())
# sorted_value_index = np.argsort(values)
# sorted_dict = {keys[i]: values[i] for i in sorted_value_index}

# #print(sorted_dict)
# for k,v in sorted_dict.items():
#     print(k,v)

In [ ]:
#takes 4min
info_list = qdf.quote_info_list.to_list()
revised_tag_list = []
indices_to_drop = []

revised_tagset = set() #just to count uniqe tags
for idx, tag_list in enumerate(info_list):
  tag_list = eval(tag_list)
  revised_tags = []

  for tag in tag_list:
    if tag not in rare_genres:
      revised_tags.append(tag)
      revised_tagset.add(tag)

  if len(revised_tags) == 0:
    indices_to_drop.append(idx)
  else:
    revised_tag_list.append(revised_tags)

qdf = qdf.drop(indices_to_drop).reset_index(drop=True)
qdf.shape

(38560, 4)

In [ ]:
len(revised_tagset)

103

In [ ]:
qdf['revised_quote_info'] = revised_tag_list

In [ ]:
qdf.tail()

In [ ]:
info_list = qdf.revised_quote_info.to_list()
info_list

revised_tag_count = {}

for tag_list in info_list: # type(info_list) is string. tag_list in info_list is string
    #print(tag_list)
    tag_list = eval(tag_list) #string type cannot be made to_list() | eval() will make string to python expression.
    for tag in tag_list:
        if tag in revised_tag_count.keys():
            revised_tag_count[tag] += 1
        else:
            revised_tag_count[tag] = 1
print(f"Number of tags: {len(revised_tag_count)}")
print(revised_tag_count)     


Number of tags: 103
{'humor': 3858, 'philosophy': 1459, 'science': 3027, 'books': 350, 'life': 8309, 'work': 329, 'writing': 353, 'religion': 964, 'women': 263, 'death': 3171, 'fear': 395, 'reading': 270, 'lies': 244, 'truth': 3598, 'people': 330, 'love': 4962, 'inspirational': 4845, 'Cassandra Clare': 304, 'god': 3202, 'funny': 341, 'humanity': 318, 'wisdom': 3544, 'success': 3075, 'knowledge': 598, 'happiness': 3327, 'faith': 779, 'time': 3009, 'friendship': 390, 'romance': 490, 'peace': 387, 'quotes': 946, 'beauty': 348, 'money': 396, 'poetry': 732, 'change': 450, 'relationships': 496, 'motivation': 1542, 'art': 374, 'dreams': 550, 'optimism': 395, 'christianity': 315, 'spirituality': 780, 'mind': 246, 'passion': 253, 'life-lessons': 3392, 'inspiration': 1405, 'education': 291, 'pain': 316, 'failure': 384, 'belief': 286, 'life-and-living': 386, 'reality': 357, 'action': 245, 'future': 344, 'nature': 399, 'living': 609, 'positive': 363, 'inspirational-life': 392, 'inspirational-quote

In [ ]:
# print(qdf.shape)
# qdf.to_csv('quote-nlp-dataset_cleaned.csv',encoding='utf-8-sig', index=False) #encoding='utf-8-sig' should work. also encoding='utf-16' but not alawys works
# qdf.tail()

encode:

In [ ]:
import json
encode_tag_types = { key: idx for idx, (key, value) in enumerate(revised_tag_count.items())}
with open("tag_types_encoded.json", "w") as fp:
  json.dump(encode_tag_types, fp)

In [ ]:
# We need this because for multilabel classification all genres have possibility to be present in the predictions
categorical_tag_list = []
revised_quote_info = qdf.revised_quote_info.to_list()
#type(df.revised_genres.to_list()), tpye(df.revised_genres)

for tag_list in revised_quote_info:
  categorical_list = [0] * len(encode_tag_types)
  tag_list = eval(tag_list)
  for tag in tag_list:
    #print(tag)
    tag_type_index = encode_tag_types[tag]
    categorical_list[tag_type_index] = 1
  categorical_tag_list.append(categorical_list)

categorical_tag_list[3][:23]

[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
qdf['tag_cat_list'] = categorical_tag_list
qdf.head()

,attirbuted author,quote,quote_info_list,quote_info_list_w_author,revised_quote_info,tag_cat_list
0,Albert Einstein,Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.,"['attributed-no-source', 'human-nature', 'humor', 'infinity', 'philosophy', 'science', 'stupidity', 'universe', 'Albert Einstein']","['attributed-no-source', 'human-nature', 'humor', 'infinity', 'philosophy', 'science', 'stupidity', 'universe', 'Albert Einstein']","['humor', 'philosophy', 'science']","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
1,Frank Zappa,"So many books, so little time.","['books', 'humor', 'Frank Zappa']","['books', 'humor', 'Frank Zappa']","['books', 'humor']","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
2,Mae West,"You only live once, but if you do it right, once is enough.","['humor', 'life', 'Mae West']","['humor', 'life', 'Mae West']","['humor', 'life']","[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
3,Narcotics Anonymous,"Insanity is doing the same thing, over and over again, but expecting different results.","['humor', 'insanity', 'life', 'misattributed-ben-franklin', 'misattributed-mark-twain', 'misattributed-to-einstein', 'Narcotics Anonymous']","['humor', 'insanity', 'life', 'misattributed-ben-franklin', 'misattributed-mark-twain', 'misattributed-to-einstein', 'Narcotics Anonymous']","['humor', 'life']","[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
4,Jane Austen,"The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.","['aliteracy', 'books', 'classic', 'humor', 'Jane Austen']","['aliteracy', 'books', 'classic', 'humor', 'Jane Austen']","['books', 'humor']","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"


# checkpoint

In [ ]:
print(qdf.shape)
qdf.to_csv('quote-nlp-dataset_cleaned.csv',encoding='utf-8-sig', index=False) #encoding='utf-8-sig' should work. also encoding='utf-16' but does not alawys work

(38560, 6)
